In [ ]:
import pandas as pd
import numpy as np
import os
import random
import time
import json
from datetime import datetime
import threading

In [ ]:
os.environ['GGS_DIR'] = '/kaggle/input/gitaggle-settings/gitaggle-settings'
os.environ['USER_NAME'] = 'Dinh Thoai Tran @ randrise.com'
os.environ['USER_EMAIL'] = 'dinhtt@randrise.com'
os.environ['LG_REPO_URL'] = 'https://github.com/dinhtt-randrise/gitaggle-ec-logger.git'
os.environ['NB_USER'] = 'dinhttrandrise'
os.environ['NB_CODE'] = 'gg-environment-controller-b'
os.environ['NB_CODE_NEXT'] = 'gg-environment-controller-a'
os.environ['DEBUG_ON'] = 'no'
USER_NAME = os.environ.get('USER_NAME', 'Dinh Thoai Tran @ randrise.com')
USER_EMAIL = os.environ.get('USER_EMAIL', 'dinhtt@randrise.com')
NB_USER = os.environ.get('NB_USER', 'dinhttrandrise')
NB_CODE = os.environ.get('NB_CODE', 'gg-environment-controller-a')
NB_CODE_NEXT = os.environ.get('NB_CODE_NEXT', 'gg-environment-controller-b')
DEBUG_ON = os.environ.get('DEBUG_ON', 'no') == 'yes'
LG_REPO_URL = os.environ.get('LG_REPO_URL', 'https://github.com/dinhtt-randrise/gitaggle-ec-logger.git')
LG_ID = 0

In [ ]:
!git clone https://github.com/dinhtt-randrise/gitaggle.git

In [ ]:
import gitaggle.logger as gglg
import gitaggle.kaggle as ggkg
import gitaggle.controller as ggcl

In [ ]:
RUN_TIME = 60 * 60 * 11
LOG_TIME = 60 * 15
SLEEP_TIME = 30
START_TIME = time.time()
CRON_TAB_TIME = 60 * 15
CRON_TAB = None
CRON_TAB_START_TIME = time.time()

In [ ]:
def run_batch(batch_no):
    global CRON_TAB_START_TIME, CRON_TAB
    try:
        need_updated = False
        if CRON_TAB is None:
            need_updated = True
        if time.time() - CRON_TAB_START_TIME > CRON_TAB_TIME:
            need_updated = True
            CRON_TAB_START_TIME = time.time()
        if need_updated:
            js = gglg.read_log_file(LG_ID, '/crontab.json', '[]')
            CRON_TAB = json.loads(js)
        for ct in CRON_TAB:
            now = datetime.now()
            cnt = 0
            if 'h' in ct:
                h = ct['h']
                if h == '*':
                    cnt += 1
                elif h == int(now.strftime("%H")):
                    cnt += 1
            else:
                cnt += 1
            if 'm' in ct:
                m = ct['m']
                if m == '*':
                    cnt += 1
                elif m == int(now.strftime("%M")):
                    cnt += 1
            else:
                cnt += 1            
            if cnt != 2:
                continue
            if 'nb_code' not in ct or 'nb_user' not in ct:
                continue
            if 'pseudo' in ct and ct['pseudo'] == 'yes':
                gglg.save_log(LG_ID, '=> [CT] ' + str(ct['nb_user']) + '/' + str(ct['nb_code']))
            else:
                try:
                    threading.Thread(target=run_notebook, args=(ct['nb_code'], ct['nb_user'],)).start()
                except Exception as ec:
                    print('=> [E] ' + str(ec))
    except Exception as e:
        print('=> [E] ' + str(e))
        
def run_notebook(nb_code, nb_user):
    try:
        nb_dir = '/kaggle/working/nb_' + str(random.randint(0, 1000000))
        os.system('mkdir -p "' + nb_dir + '"')
        ggkg.download_notebook(nb_code, nb_dir, nb_user)
        meta_file = nb_dir + '/kernel-metadata.json'
        lines = ggkg.read_text_file(meta_file)
        lines = lines.replace('"enable_internet": false,', '"enable_internet": true,')
        ggkg.write_text_file(meta_file, lines)
        cmd = 'kaggle kernels push -p "' + nb_dir + '"'
        if DEBUG_ON:
            print('=> [CMD] ' + cmd)
        os.system(cmd)    
        os.system('rm -rf "' + nb_dir + '"')
    except Exception as e:
        print('=> [E] ' + str(e))

In [ ]:
LG_ID = gglg.create_logger(USER_NAME, USER_EMAIL, LG_REPO_URL, NB_CODE)
gglg.save_log(LG_ID, '[' + NB_CODE + '] is started!')
batch_no = 0
start_time = time.time()
while time.time() - START_TIME < RUN_TIME:
    batch_no += 1
    run_batch(batch_no)
    if time.time() - start_time > LOG_TIME:
        try:
            gglg.flush_all()
        except Exception as e:
            print('=> [E] ' + str(e))
        start_time = time.time()
    time.sleep(SLEEP_TIME)

try:
    gglg.flush_all()
except Exception as e:
    print('=> [E] ' + str(e))
    
try:
    nb_dir = '/kaggle/working/nb_next'
    os.system('mkdir -p "' + nb_dir + '"')
    ggkg.download_notebook(NB_CODE_NEXT, nb_dir, NB_USER)
    meta_file = nb_dir + '/kernel-metadata.json'
    lines = ggkg.read_text_file(meta_file)
    lines = lines.replace('"enable_internet": false,', '"enable_internet": true,')
    ggkg.write_text_file(meta_file, lines)
    cmd = 'kaggle kernels push -p "' + nb_dir + '"'
    if DEBUG_ON:
        print('=> [CMD] ' + cmd)
    os.system(cmd)    
    os.system('rm -rf "' + nb_dir + '"')
except Exception as e:
    print('=> [E] ' + str(e))